Dataset: https://archive.ics.uci.edu/ml/datasets/wine+quality \
Project predicts quality of wine basing on it's parameters. To select best model, Azure Automated Machine Learning service was used. \
First of all, there was carried out test of that service using portal: https://ml.azure.com/ with following steps:
- create workspace
- create compute cluster with name "AiOnAzureCluster"
- create dataset, by uploading it from computer
- run Automated ML Experiment 
Best model was selected and results was quite good.

After previous steps computing cluster and dataset was already created, so in this notebook they are only loaded.

In [1]:
print("setting of names variables")
dataset_name = 'Wine'
amlcompute_cluster_name = "AiOnAzureCluster"
predicting_column_name = 'quality'

project_folder = './wine-automl'
os.makedirs(project_folder, exist_ok=True)

setting of names variables


In [2]:
from azureml.core import Workspace, Dataset

print("loading azure ml workspace and dataset")
ws = Workspace.from_config()
aml_dataset = ws.datasets[dataset_name]
full_df = aml_dataset.to_pandas_dataframe()

loading azure ml workspace and dataset


In [3]:
print("dataset characteristics")
full_df.describe()

dataset characteristics


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1595.000000,1597.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.838871,46.428929,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.423696,32.897570,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [4]:
print("splitting dataset into train and test")
train_dataset, test_dataset = aml_dataset.random_split(0.9, seed=5)

train_dataset_df = train_dataset.to_pandas_dataframe()
test_dataset_df = test_dataset.to_pandas_dataframe()

train_dataset_df.describe()

splitting dataset into train and test


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1402.000000,1402.000000,1402.000000,1402.000000,1402.000000,1399.000000,1401.000000,1402.000000,1402.000000,1402.000000,1402.000000,1402.000000
mean,8.334736,0.528181,0.271605,2.537518,0.088106,15.989278,47.049251,0.996790,3.310899,0.661084,10.404743,5.631241
std,1.738866,0.180044,0.194177,1.397057,0.048025,10.476107,32.762714,0.001859,0.154512,0.172484,1.053730,0.798085
min,4.600000,0.120000,0.000000,0.900000,0.034000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.100000,1.900000,0.071000,7.500000,23.000000,0.995650,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.080000,14.000000,38.000000,0.996800,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.635000,0.430000,2.600000,0.091000,22.000000,63.000000,0.997877,3.400000,0.730000,11.000000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

print("loading compute cluster. Cluster should be type 'AmlCompute'.")
targets = ws.compute_targets
if amlcompute_cluster_name in targets and targets[amlcompute_cluster_name].type == 'AmlCompute':
    aml_remote_compute = targets[amlcompute_cluster_name]
    aml_remote_compute.wait_for_completion(show_output = True, min_node_count = 0, timeout_in_minutes = 20)
else:
    print(f'ERROR - no cluster with name "{amlcompute_cluster_name}" found')
    

loading compute cluster. Cluster should be type 'AmlCompute'.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azureml.core import Experiment
from azureml.train.automl import AutoMLConfig
from datetime import datetime
import logging
import os

print("setting azure ML experiment - using regression and spearman_correlation as a metric.")
automl_config = AutoMLConfig(compute_target=aml_remote_compute,
                             task='regression',
                             primary_metric='spearman_correlation',
                             experiment_timeout_minutes=15,
                             training_data=train_dataset,
                             label_column_name=predicting_column_name,
                             n_cross_validations=5,
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log='automated_ml_errors.log',
                             verbosity=logging.INFO,
                             path=project_folder
                             )

print("Creating experiment with unique name and starting it")
experiment_name = f'regr-aml-wine-{datetime.now().strftime("%m-%d-%Y-%H")}'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(automl_config, show_output=True)

setting azure ML experiment - using regression and spearman_correlation as a metric.
Creating experiment with unique name and starting it
Running on remote.
No run_configuration provided, running on AiOnAzureCluster with default configuration
Running on remote compute: AiOnAzureCluster
Parent Run ID: AutoML_bb64e09a-366e-4a7d-9a60-6dfcf2bc16e9

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business req

In [7]:
from azureml.widgets import RunDetails

print("Displaying details of the run. Every model has it's own result for many metrics")
RunDetails(run).show()

Displaying details of the run. Every model has it's own result for many metrics


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [8]:
print("Preparing test dataset to measure performance")
y_test_df = test_dataset_df.pop(predicting_column_name)
x_test_df = test_dataset_df

print("getting best model and getting predictions for test dataset")
_, best_model = run.get_output()
y_hat = best_model.predict(x_test_df)

Preparing test dataset to measure performance
getting best model and getting predictions for test dataset


In [9]:
from sklearn.metrics import mean_squared_error, r2_score

print("calculating and displaying best model performance:")
print('MSE:')
print(mean_squared_error(y_test_df, y_hat))
print('R2:')
print(r2_score(y_test_df, y_hat))

calculating and displaying best model performance:
MSE:
0.4430247264380246
R2:
0.4164229648926313
